# Blinks, Mouth Size Change, Emotion 측정

In [1]:
!pip install mediapipe
!pip install opencv-python
!pip install deepface
!pip insttall numpy

^C


In [ ]:
import cv2
# from google.colab.patches import cv2_imshow
import mediapipe as mp
import numpy as np
from deepface import DeepFace

# EAR 계산 함수
def calculate_ear(landmarks, indices):
    # 두 점 간의 유클리드 거리 계산
    def euclidean_dist(point1, point2):
        return np.linalg.norm(np.array(point1) - np.array(point2))

    # EAR 계산 공식
    A = euclidean_dist(landmarks[indices[1]], landmarks[indices[5]])
    B = euclidean_dist(landmarks[indices[2]], landmarks[indices[4]])
    C = euclidean_dist(landmarks[indices[0]], landmarks[indices[3]])
    ear = (A + B) / (2.0 * C)
    return ear

# MAR 계산 함수
def mouth_aspect_ratio(mouth_coords):
    # 주요 점들을 선택 (입의 상하/좌우 랜드마크)
    upper_lip_top = np.array(mouth_coords[2])  # 위쪽 중앙 (mouth_indices[13])
    lower_lip_bottom =  np.array(mouth_coords[3])  # 아래쪽 중앙 (mouth_indices[14])
    upper_lip_bottom =  np.array(mouth_coords[4])  # 위쪽 안쪽 (mouth_indices[312])
    lower_lip_top =  np.array(mouth_coords[5])  # 아래쪽 안쪽 (mouth_indices[82])
    left_corner =  np.array(mouth_coords[0])  # 왼쪽 모서리 (mouth_indices[78])
    right_corner =  np.array(mouth_coords[1])  # 오른쪽 모서리 (mouth_indices[308])

    # 상하 거리 계산
    vertical_distance_1 = np.linalg.norm(upper_lip_top - lower_lip_bottom)
    vertical_distance_2 = np.linalg.norm(upper_lip_bottom - lower_lip_top)

    # 좌우 거리 계산
    horizontal_distance = np.linalg.norm(left_corner - right_corner)

    # MAR 계산
    mar = (vertical_distance_1 + vertical_distance_2) / (2.0 * horizontal_distance)
    return mar

# 감정 분석 함수
def analyze_emotion(face_frame):
    try:
        analysis = DeepFace.analyze(face_frame, actions=["emotion"], enforce_detection=False)
        # print(analysis)  # analysis 출력 추가
        return analysis[0]["dominant_emotion"]  # 수정된 부분: 리스트 형식으로 반환됨
    except Exception as e:
        print(f"DeepFace Error: {e}")
        return "Unknown"

def extract_facial_features_with_emotion():
    # Mediapipe 초기화
    mp_face_mesh = mp.solutions.face_mesh
    mp_drawing = mp.solutions.drawing_utils
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

    # 비디오 파일 읽기
    cap = cv2.VideoCapture('face_example.mp4')

    blink_count = 0
    ear_threshold = 0.2
    ear_consecutive_frames = 2
    frame_counter = 0

    mouth_size_changes = []
    prev_mar = None

    emotion_counts = {"happy": 0, "sad": 0, "angry": 0, "fear": 0, "surprise": 0, "neutral": 0, "disgust": 0, "Unknown": 0}

    # EYE_CLOSED_THRESHOLD = 3  # 눈 감은 상태 지속 프레임 임계값
    # EAR_THRESHOLD = 0.2  # 눈 종횡비(eye aspect ratio) 임계값
    # eye_closed_frames = 0

    # frame_count = 0  # 프레임 카운트 변수
    # fps = cap.get(cv2.CAP_PROP_FPS)  # FPS 값 얻기
    # frame_interval = int(fps)  # 1초에 한 번씩 분석

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # frame_count += 1  # 프레임 카운트 증가

        # 이미지 전처리
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        face_cropped = None

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 얼굴 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                    mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1))

                # 얼굴 영역 추출 (DeepFace 입력용)
                h, w, _ = frame.shape
                x_min = y_min = 1
                x_max = y_max = 0

                # 얼굴 랜드마크의 최소/최대 좌표 계산
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    x_min = min(x_min, x)
                    y_min = min(y_min, y)
                    x_max = max(x_max, x)
                    y_max = max(y_max, y)

                # 얼굴 영역 자르기
                face_cropped = frame[y_min:y_max, x_min:x_max]

                # 좌표 변환
                landmarks = [(int(lm.x * w), int(lm.y * h)) for lm in face_landmarks.landmark]

                # EAR 계산 (눈 깜빡임 감지)
                left_eye_indices = [33, 160, 158, 133, 153, 144]
                right_eye_indices = [362, 385, 387, 263, 373, 380]
                left_ear = calculate_ear(landmarks, left_eye_indices)
                right_ear = calculate_ear(landmarks, right_eye_indices)
                ear = (left_ear + right_ear) / 2.0

                # 눈 깜빡임 감지
                if ear < ear_threshold:
                    frame_counter += 1
                else:
                    if frame_counter >= ear_consecutive_frames:
                        blink_count += 1
                    frame_counter = 0

                # 입 MAR 계산
                mouth_indices = [78, 308, 13, 14, 312, 82]
                mouth_coords = [landmarks[i] for i in mouth_indices]
                mar = mouth_aspect_ratio(mouth_coords)
                if prev_mar is not None:
                    mouth_size_changes.append(abs(mar - prev_mar))
                prev_mar = mar

        # 1초마다 감정 분석
        # if frame_count % frame_interval == 0:
          # DeepFace 감정 분석
        if face_cropped is not None:
            emotion = analyze_emotion(face_cropped)
            emotion_counts[emotion] += 1
            cv2.putText(frame, f"Emotion: {emotion}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # EAR, 눈 깜빡임, MAR, 감정 상태 출력
        cv2.putText(frame, f'EAR: {ear:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Blinks: {blink_count}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # 비디오 출력 (1초마다 출력)
        # 비디오 출력
        cv2.imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()
    avg_mouth_size_change = np.mean(mouth_size_changes) if mouth_size_changes else 0
    return blink_count, avg_mouth_size_change, emotion_counts

# 실행
blink_count, avg_mouth_size_change, emotion_counts = extract_facial_features_with_emotion()
print(f"Total Blinks: {blink_count}")
print(f"Average Mouth Size Change: {avg_mouth_size_change}")
print(f"Emotion Summary: {emotion_counts}")